🔥 **High-Impact Business Questions:**

🚨 **Immediate Fraud Flags:**
 → Which transaction types and payment channels have the highest fraud rates? 💳📊

🧠 **Behavioral Analysis:**
 → Do fraudulent transactions have a higher `velocity_score` and `time_since_last_transaction` close to 0? ⚡⏱️
  (Are they sudden, rapid transactions?)

🌍 **Geographic & Device Risk:**
 → Is there a correlation between high `geo_anomaly_score` and confirmed fraud? 🗺️📱

🏷️ **Fraud Category:**
 → What is the most common type of fraud (`fraud_type`) we are experiencing? 🕵️‍♂️📂

🏪 **Merchant Risk:**
 → Which `merchant_category` has the highest frequency of fraud? 🏦💰

 

In [1]:
import sqlite3
import pandas as pd 

In [2]:

# read the CSV
df = pd.read_csv('financial_fraud_detection_dataset.csv')

# connect to SQLite (in memory or save as file)
conn = sqlite3.connect(':memory:')   # use ':memory:' or 'fraud.db'
df.to_sql('transactions', conn, index=False, if_exists='replace')


5000000

In [3]:
df.columns

Index(['transaction_id', 'timestamp', 'sender_account', 'receiver_account',
       'amount', 'transaction_type', 'merchant_category', 'location',
       'device_used', 'is_fraud', 'fraud_type', 'time_since_last_transaction',
       'spending_deviation_score', 'velocity_score', 'geo_anomaly_score',
       'payment_channel', 'ip_address', 'device_hash'],
      dtype='object')

✅ **What You’re Doing Well**

* You’re validating data (`.head()`, `.isnull().sum()`).
* You’re cleaning nulls responsibly.
* You’re linking pandas with SQL for inspection.


In [4]:
df.head()

,transaction_id,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,fraud_type,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash
0,T100000,2023-08-22T09:22:43.516168,ACC877572,ACC388389,343.78,withdrawal,utilities,Tokyo,mobile,False,NaN,NaN,-0.21,3,0.22,card,13.101.214.112,D8536477
1,T100001,2023-08-04T01:58:02.606711,ACC895667,ACC944962,419.65,withdrawal,online,Toronto,atm,False,NaN,NaN,-0.14,7,0.96,ACH,172.52.47.194,D2622631
2,T100002,2023-05-12T11:39:33.742963,ACC733052,ACC377370,2773.86,deposit,other,London,pos,False,NaN,NaN,-1.78,20,0.89,card,185.98.35.23,D4823498
3,T100003,2023-10-10T06:04:43.195112,ACC996865,ACC344098,1666.22,deposit,online,Sydney,pos,False,NaN,NaN,-0.60,6,0.37,wire_transfer,107.136.36.87,D9961380
4,T100004,2023-09-24T08:09:02.700162,ACC584714,ACC497887,24.43,transfer,utilities,Toronto,mobile,False,NaN,NaN,0.79,13,0.27,ACH,108.161.108.255,D7637601


In [5]:
df.isnull().sum()

transaction_id                       0
timestamp                            0
sender_account                       0
receiver_account                     0
amount                               0
transaction_type                     0
merchant_category                    0
location                             0
device_used                          0
is_fraud                             0
fraud_type                     4820447
time_since_last_transaction     896513
spending_deviation_score             0
velocity_score                       0
geo_anomaly_score                    0
payment_channel                      0
ip_address                           0
device_hash                          0
dtype: int64

In [6]:
df['fraud_type'] = df['fraud_type'].fillna('not_fraud')
df['time_since_last_transaction'] = df['time_since_last_transaction'].fillna(0) 

In [7]:
df.isnull().sum()

transaction_id                 0
timestamp                      0
sender_account                 0
receiver_account               0
amount                         0
transaction_type               0
merchant_category              0
location                       0
device_used                    0
is_fraud                       0
fraud_type                     0
time_since_last_transaction    0
spending_deviation_score       0
velocity_score                 0
geo_anomaly_score              0
payment_channel                0
ip_address                     0
device_hash                    0
dtype: int64

In [8]:
pd.read_sql_query("SELECT * FROM transactions LIMIT 5;", conn)

,transaction_id,timestamp,sender_account,receiver_account,amount,transaction_type,merchant_category,location,device_used,is_fraud,fraud_type,time_since_last_transaction,spending_deviation_score,velocity_score,geo_anomaly_score,payment_channel,ip_address,device_hash
0,T100000,2023-08-22T09:22:43.516168,ACC877572,ACC388389,343.78,withdrawal,utilities,Tokyo,mobile,0,None,None,-0.21,3,0.22,card,13.101.214.112,D8536477
1,T100001,2023-08-04T01:58:02.606711,ACC895667,ACC944962,419.65,withdrawal,online,Toronto,atm,0,None,None,-0.14,7,0.96,ACH,172.52.47.194,D2622631
2,T100002,2023-05-12T11:39:33.742963,ACC733052,ACC377370,2773.86,deposit,other,London,pos,0,None,None,-1.78,20,0.89,card,185.98.35.23,D4823498
3,T100003,2023-10-10T06:04:43.195112,ACC996865,ACC344098,1666.22,deposit,online,Sydney,pos,0,None,None,-0.60,6,0.37,wire_transfer,107.136.36.87,D9961380
4,T100004,2023-09-24T08:09:02.700162,ACC584714,ACC497887,24.43,transfer,utilities,Toronto,mobile,0,None,None,0.79,13,0.27,ACH,108.161.108.255,D7637601


🚨 Immediate Fraud Flags:
 → Which transaction types and payment channels have the highest fraud rates? 💳📊

In [9]:
q1 = """
SELECT
  transaction_type,
  payment_channel,
  COUNT(*) AS txns,
  SUM(CASE WHEN is_fraud = 1 THEN 1 ELSE 0 END) AS fraud_txns,
  ROUND(AVG(CASE WHEN is_fraud = 1 THEN 1.0 ELSE 0 END), 4) AS fraud_rate
FROM transactions
GROUP BY transaction_type, payment_channel
ORDER BY fraud_rate DESC, fraud_txns DESC;
"""
df_q1 = pd.read_sql_query(q1, conn)
df_q1

,transaction_type,payment_channel,txns,fraud_txns,fraud_rate
0,payment,wire_transfer,312889,11409,0.0365
1,transfer,UPI,311888,11396,0.0365
2,transfer,card,313192,11351,0.0362
3,transfer,ACH,312364,11315,0.0362
4,withdrawal,UPI,312231,11293,0.0362
5,transfer,wire_transfer,312890,11266,0.0360
6,withdrawal,ACH,312577,11263,0.0360
7,deposit,card,312379,11258,0.0360
8,withdrawal,wire_transfer,312360,11188,0.0358
9,deposit,UPI,312515,11188,0.0358


💳 Highest fraud rate: payment via wire_transfer (≈ 3.65 %)

🔁 Transfers & withdrawals are close behind (≈ 3.6 %).

🪙 Overall pattern: wire transfers and UPI channels are slightly riskier than cards or ACH. 

Next step 👉 Step 2 – Behavioral Analysis

Run this to compare average velocity and time gap for fraud vs non-fraud:

In [10]:
q2 = """
SELECT
  is_fraud,
  ROUND(AVG(velocity_score),2) AS avg_velocity,
  ROUND(AVG(time_since_last_transaction),2) AS avg_time_gap
FROM transactions
GROUP BY is_fraud;
"""
df_q2 = pd.read_sql_query(q2, conn)
df_q2 

,is_fraud,avg_velocity,avg_time_gap
0,0,10.50,1.51
1,1,10.51,1.77


Velocity score:
→ Fraud (10.51) ≈ Legit (10.50)
→ ❌ No clear difference. Fraud isn’t faster in this dataset.

Time since last transaction:
→ Fraud = 1.77 vs Legit = 1.51
→ ⚖️ Slightly longer gap before fraud, meaning it’s not “sudden” — the frauds happen after a bit more time, not right after the last transaction.


✅ Answer:
Fraudulent transactions do not appear more sudden or rapid. Both fraud and normal transactions have almost identical behavior patterns based on these metrics.



🌍 Step 3 – Geographic & Device Risk

In [11]:
df_geo = pd.read_sql_query("SELECT geo_anomaly_score, is_fraud FROM transactions;", conn)
corr_geo_fraud = df_geo.corr(numeric_only=True).loc["geo_anomaly_score", "is_fraud"]
corr_geo_fraud
q3b = """
WITH t AS (
  SELECT
    CASE WHEN geo_anomaly_score >= 0.8 THEN 'high_geo' ELSE 'normal_geo' END AS geo_band,
    is_fraud
  FROM transactions
)
SELECT
  geo_band,
  ROUND(AVG(CASE WHEN is_fraud = 1 THEN 1.0 ELSE 0 END), 4) AS fraud_rate,
  COUNT(*) AS txns
FROM t
GROUP BY geo_band
ORDER BY fraud_rate DESC;
"""
df_q3b = pd.read_sql_query(q3b, conn)
df_q3b

,geo_band,fraud_rate,txns
0,normal_geo,0.0359,3975663
1,high_geo,0.0359,1024337


In [12]:
corr_pearson  = df.corr(numeric_only=True)              # Pearson correlation
corr_spearman = df.corr(numeric_only=True, method="spearman")  # Spearman correlation

# Correlation of each column with the target (is_fraud)
corr_pearson['is_fraud'].sort_values(ascending=False)
corr_spearman['is_fraud'].sort_values(ascending=False)

is_fraud                       1.000000
velocity_score                 0.000370
geo_anomaly_score              0.000311
spending_deviation_score       0.000178
time_since_last_transaction   -0.000047
amount                        -0.000185
Name: is_fraud, dtype: float64

All correlation values are extremely close to 0, meaning. In other words, fraud can’t be easily predicted using just these numeric columns.


In [13]:
q5_count = """
SELECT
  merchant_category,
  SUM(CASE WHEN is_fraud = 1 THEN 1 ELSE 0 END) AS fraud_txns,
  COUNT(*) AS total_txns
FROM transactions
GROUP BY merchant_category
ORDER BY fraud_txns DESC;
"""
df_q5_count = pd.read_sql_query(q5_count, conn)
df_q5_count.head(10)

,merchant_category,fraud_txns,total_txns
0,entertainment,22573,625332
1,other,22556,624589
2,grocery,22516,624954
3,travel,22503,625656
4,retail,22453,626319
5,restaurant,22367,625483
6,online,22324,623581
7,utilities,22261,624086


In [14]:
q5_rate = """
SELECT
  merchant_category,
  ROUND(AVG(CASE WHEN is_fraud = 1 THEN 1.0 ELSE 0 END), 4) AS fraud_rate,
  COUNT(*) AS txns
FROM transactions
GROUP BY merchant_category
HAVING COUNT(*) >= 20
ORDER BY fraud_rate DESC, txns DESC;
"""
df_q5_rate = pd.read_sql_query(q5_rate, conn)
df_q5_rate.head(10)

,merchant_category,fraud_rate,txns
0,entertainment,0.0361,625332
1,other,0.0361,624589
2,travel,0.0360,625656
3,grocery,0.0360,624954
4,retail,0.0358,626319
5,restaurant,0.0358,625483
6,online,0.0358,623581
7,utilities,0.0357,624086


💡 Interpretation

Fraud rates are very similar (~3.6 %) across all merchant categories.

🎭 Entertainment and 💼 Other are just slightly higher, but the difference is tiny — less than 0.0003.

➜ This means merchant type doesn’t strongly affect fraud risk in this dataset.

Perfect 👌 — here’s your full short summary for **Phases 1–5** of analysis (SQL + pandas).

---

## 🧾 **Fraud Detection Analysis Summary**

### 🚨 1. Immediate Fraud Flags

**Question:** Which transaction types and payment channels have the highest fraud rates?
**Finding:**

* 💳 *Payment via wire transfer* had the **highest fraud rate (~3.65%)**.
* 🪙 *Transfers* and *withdrawals* were also near that level.
* ✅ Wire transfer and UPI are **slightly riskier** channels.

---

### 🧠 2. Behavioral Analysis

**Question:** Do fraudulent transactions have higher velocity or shorter time gaps?
**Finding:**

* Fraud: avg velocity = **10.51**, avg gap = **1.77**
* Non-fraud: avg velocity = **10.50**, avg gap = **1.51**
* ⚖️ Result: **No major behavioral difference** — frauds are not more sudden or rapid.

---

### 🌍 3. Geographic & Device Risk

**Question:** Is high geo_anomaly_score linked to fraud?
**Finding:**

* Correlation ≈ **0.00** → No relationship.
* Fraud rate: **Normal geo = 3.59%**, **High geo = 3.59%**
* 📊 Visual confirmed both bars equal height.
* ✅ No location-based fraud pattern.

---

### 🏷️ 4. Fraud Category

**Finding:**

* Only **one fraud type:** `card_not_present`.
* ⚙️ All frauds are of this single type → dataset is **focused on that scenario only.**

---

### 🏪 5. Merchant Risk

**Question:** Which merchant categories have highest fraud?
**Finding:**

| Top Categories   | Fraud Rate |
| ---------------- | ---------- |
| 🎭 Entertainment | 3.61%      |
| 💼 Other         | 3.61%      |
| ✈️ Travel        | 3.60%      |
| 🛒 Grocery       | 3.60%      |
| 🏬 Retail        | 3.58%      |

* 📉 All are nearly identical → **no single merchant group dominates fraud activity.**

---

### 🧠 Overall Summary

* 🔍 **Fraud rate** consistent around **3.6%** across all dimensions.
* ⚙️ **No strong numeric correlation** with fraud (`velocity_score`, `geo_anomaly_score`, etc. ≈ 0).
* 💳 Fraud type = *card_not_present* → suggests this dataset is simulating credit card fraud scenarios.




